In [20]:
import numpy as np
import pandas as pd
import sys
import datasets
import matplotlib
import openpyxl
from nltk.tokenize import RegexpTokenizer

##### Compare all_code file of SELF and PLM data

In [21]:
"""
all_ours = pd.read_csv('./PLM-ICD-master/data/mimic3/ALL_ICD_CODES.txt', header = None)[0]
all_plm = pd.read_csv("./PLM-ICD-master/data/mimic3/ALL_CODES.txt", header = None)[0]
print('The number of codes in ours and plm all_code files are: ', len(all_ours),' and ',len(all_plm))
"""

'\nall_ours = pd.read_csv(\'./PLM-ICD-master/data/mimic3/ALL_ICD_CODES.txt\', header = None)[0]\nall_plm = pd.read_csv("./PLM-ICD-master/data/mimic3/ALL_CODES.txt", header = None)[0]\nprint(\'The number of codes in ours and plm all_code files are: \', len(all_ours),\' and \',len(all_plm))\n'

In [22]:
"""
print(sum(1-all_ours.isin(all_plm)), 'of our codes are not in plm all_code')
print(sum(1-all_plm.isin(all_ours)), 'of plm codes are not in our all_code')
"""

"\nprint(sum(1-all_ours.isin(all_plm)), 'of our codes are not in plm all_code')\nprint(sum(1-all_plm.isin(all_ours)), 'of plm codes are not in our all_code')\n"

In [23]:

#################### process SELF curated data
all_code_txt = './PLM-ICD-master/data/mimic3/ALL_ICD_CODES.txt'
train_csv = 'PLM-ICD-master/data/mimic3/train_notes_pv1_full_trimmed.csv'
dev_csv = 'PLM-ICD-master/data/mimic3/dev_notes_pv1_full_trimmed.csv'
test_csv = 'PLM-ICD-master/data/mimic3/test_notes_pv1_full_trimmed.csv'
path = 'PLM-ICD-master/data/mimic3'

text_colname = 'text'
label_colname = 'label'


In [24]:
"""
#################### process CAML curated data
all_code_txt = "./PLM-ICD-master/data/mimic3/ALL_CODES.txt"
train_csv = "./caml-mimic-master/mimicdata/mimic3/train_full_trimmed.csv"
dev_csv = "./caml-mimic-master/mimicdata/mimic3/dev_full_trimmed.csv"
test_csv = "./caml-mimic-master/mimicdata/mimic3/test_full_trimmed.csv"
path = "./caml-mimic-master/mimicdata/mimic3"

#text_colname = 'TEXT'
#label_colname = 'LABELS'
text_colname = 'text'
label_colname = 'label'
"""

'\n#################### process CAML curated data\nall_code_txt = "./PLM-ICD-master/data/mimic3/ALL_CODES.txt"\ntrain_csv = "./caml-mimic-master/mimicdata/mimic3/train_full_trimmed.csv"\ndev_csv = "./caml-mimic-master/mimicdata/mimic3/dev_full_trimmed.csv"\ntest_csv = "./caml-mimic-master/mimicdata/mimic3/test_full_trimmed.csv"\npath = "./caml-mimic-master/mimicdata/mimic3"\n\n#text_colname = \'TEXT\'\n#label_colname = \'LABELS\'\ntext_colname = \'text\'\nlabel_colname = \'label\'\n'

##### reformat ICD codes in Code Definition File

In [25]:
def reformat(code, is_diag): ## leveraged from caml-mimic (for adding periods (dots) to ICD codes)
    """
        Put a period in the right place because the MIMIC-3 data files exclude them.
        Generally, procedure codes have dots after the first two digits, 
        while diagnosis codes have dots after the first three digits.
    """
    code = ''.join(code.split('.'))
    if is_diag:
        if code.startswith('E'):
            if len(code) > 4:
                code = code[:4] + '.' + code[4:]
        else:
            if len(code) > 3:
                code = code[:3] + '.' + code[3:]
    else:
        code = code[:2] + '.' + code[2:]
    return code
    
def tokenize_lower_nonnum(text):
    tokenizer = RegexpTokenizer(r'\w+')
    updated_text = [token.lower() for token in tokenizer.tokenize(text) if not token.isnumeric()]
    updated_text = '"'+ ' '.join(updated_text) + '"'
    return updated_text    

In [26]:
icd_dx = pd.read_excel("phecode_map/CMS32_DESC_LONG_SHORT_DX.xlsx", dtype='str')
icd_sg = pd.read_excel("phecode_map/CMS32_DESC_LONG_SHORT_SG.xlsx", dtype='str')

In [27]:
## reformat codes
icd_dx['CODE'] = icd_dx['DIAGNOSIS CODE'].apply(lambda x: reformat(str(x), True))
icd_sg['CODE'] = icd_sg['PROCEDURE CODE'].apply(lambda x: reformat(str(x), False))
icd_all = pd.concat([icd_dx[['CODE','SHORT DESCRIPTION','LONG DESCRIPTION']],icd_sg[['CODE','SHORT DESCRIPTION','LONG DESCRIPTION']]])

##### Exclude codes not used in mimic data

In [28]:
mimic3_codes = pd.read_csv(all_code_txt, header = None)[0]
temp = mimic3_codes[mimic3_codes.isin(icd_all.CODE)]
print(len(temp), 'out of', len(mimic3_codes), 'ICD9 codes are matched with descriptions.')
icd_all = icd_all[icd_all.CODE.isin(mimic3_codes)]

8593 out of 8907 ICD9 codes are matched with descriptions.


##### Combine short and long descriptions

In [29]:
icd_all['description'] = [' '.join(icd_all.iloc[i,[1, 2]]) for i in range(len(icd_all))]

In [30]:
icd_all['SUBJECT_ID']=''
icd_all['HADM_ID']=''
icd_alll = icd_all[['SUBJECT_ID', 'HADM_ID', 'description', 'CODE']]
icd_alll['description'] = icd_alll['description'].apply(lambda x: tokenize_lower_nonnum(x)) ## tokenize, lower case, remove numbers
icd_alll["length"] = icd_alll['description'].apply(lambda x: len(x.split()))
icd_alll.sort_values(by="length", inplace=True)
icd_alll.rename(columns={"CODE": label_colname, "description": text_colname}, inplace=True)
##save code description in training data format
icd_alll.to_csv(path+'/code_desc.csv', index=False)
icd_alll[0:2]

/u/x/i/xianshi/miniconda3/envs/bert_hw/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/u/x/i/xianshi/miniconda3/envs/bert_hw/lib/python3.7/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """
/u/x/i/xianshi/miniconda3/envs/bert_hw/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

,SUBJECT_ID,HADM_ID,text,label,length
9292,,,"""dysphonia dysphonia""",784.42,2
9560,,,"""hypoxemia hypoxemia""",799.02,2


In [31]:
icd_all['description'] = [''.join(['dictionary ',t]) for t in icd_all.description]
icd_all['description'] = icd_all['description'].apply(lambda x: tokenize_lower_nonnum(x)) ## tokenize, lower case, remove numbers

In [32]:
icd_all.head

<bound method NDFrame.head of        CODE         SHORT DESCRIPTION                       LONG DESCRIPTION  \
8     003.0      Salmonella enteritis             Salmonella gastroenteritis   
9     003.1     Salmonella septicemia                  Salmonella septicemia   
16    003.8  Salmonella infection NEC  Other specified salmonella infections   
17    003.9  Salmonella infection NOS      Salmonella infection, unspecified   
19    004.1         Shigella flexneri                      Shigella flexneri   
...     ...                       ...                                    ...   
3868  99.83        Other phototherapy                     Other phototherapy   
3869  99.84                 Isolation                              Isolation   
3872  99.88  Therapeutc photopheresis              Therapeutic photopheresis   
3877  99.95    Stretching of foreskin                 Stretching of foreskin   
3881  99.99    Miscellaneous proc NEC         Other miscellaneous procedures   

         

##### Add code descriptions to the training data

In [33]:
train_ds_notes = pd.read_csv(train_csv)
dev_ds_notes = pd.read_csv(dev_csv)
test_ds_notes = pd.read_csv(test_csv)

In [34]:
dffss = [train_ds_notes, dev_ds_notes, test_ds_notes]

for df in dffss:
    df = df[df[label_colname] != 'nan']
    df[text_colname] = [' '.join(['"summary', t[1:]]) for t in df[text_colname]]
    df["length"] = df[text_colname].apply(lambda x: len(x.split()))

train_ds_notes.head

<bound method NDFrame.head of        SUBJECT_ID  HADM_ID                                              label  \
0             158   169433               532.40;493.20;V45.81;412;401.9;44.43   
1            2896   178124  211.3;427.31;578.9;560.1;496;584.9;428.0;276.5...   
2            6495   139808  998.59;998.32;905.4;E929.0;041.85;86.22;86.69;...   
3            3564   117638  038.49;041.6;785.59;518.81;507.0;592.1;591;276...   
4            7995   190945               440.22;492.8;401.9;714.0;39.29;88.48   
...           ...      ...                                                ...   
47365       52139   137090  V58.11;486;205.00;277.88;038.9;995.92;785.52;4...   
47366       17488   180176  276.9;038.8;518.81;286.6;585;780.39;255.4;531....   
47367       16655   105131  320.3;996.81;428.0;599.0;038.11;421.0;995.92;4...   
47368         339   112625  577.0;995.94;574.21;518.81;584.9;482.83;511.9;...   
47369       25030   172599  431;331.4;996.81;403.91;707.0;250.81;780.39;51...  

In [35]:
icd_all.rename(columns={"CODE": label_colname, "description": text_colname}, inplace=True)

In [36]:
train_ds_notes_aug = pd.concat([train_ds_notes, icd_all[[text_colname,label_colname]]])
train_ds_notes_aug["length"] = train_ds_notes_aug[text_colname].apply(lambda x: len(x.split()))
train_ds_notes_aug.sort_values(by="length", inplace=True)

In [37]:
len(train_ds_notes_aug)
train_ds_notes_aug[0:5]

,SUBJECT_ID,HADM_ID,label,length,text
9291,NaN,NaN,784.41,3,"""dictionary aphonia aphonia"""
8693,NaN,NaN,750.16,3,"""dictionary microglossia microglossia"""
2489,NaN,NaN,272.7,3,"""dictionary lipidoses lipidoses"""
2488,NaN,NaN,272.6,3,"""dictionary lipodystrophy lipodystrophy"""
8746,NaN,NaN,752.61,3,"""dictionary hypospadias hypospadias"""


In [38]:
##save train, dev, test data to csv
train_ds_notes_aug.to_csv(train_csv[:-4]+'_aug.csv', index=False)
dev_ds_notes.to_csv(dev_csv[:-4]+'_aug.csv', index=False)
test_ds_notes.to_csv(test_csv[:-4]+'_aug.csv', index=False)